# Optimisation 101 with OR-Tools

_By Nick Brooks, February 2020_

**Resource:** <br>
- https://developers.google.com/optimization/lp/lp
- https://github.com/google/or-tools/tree/stable/examples/python
- https://docs.mosek.com/modeling-cookbook/intro.html

Big Shoutout to Google Developer Guides for putting together top notch guides! This is not my work.

## Table of Content
**Types of Optimization:**
1. Linear Optimisation
1. Contrained Opimisation
1. Integer Optimisation
1. Routing Problems

**Sample Problems:**
1. Stigler Diet Problem
1. Sodoku Problem
1. Clustering Problem
1. Zebra Problem (Lewis Caroll)


## What is Optimization?
It is the process of finding the optimal point on a function. This is what fuels most machine learning techniques. However, when people refer to just "Optimization", it is often assumed that the problem has contraints, hence **Contrained Optimization**. These introduce a lot of complexity, especially as the problem introduces more dimensions.


## Linear Optimisation
Process of optimizing a linear fucntion subject to linear contraints. The [Simplex Algorithmn](https://en.wikipedia.org/wiki/Simplex_algorithm) is the most famous of this group. This algorithmn finds an optimal solution by traversing the vertices of the feasible region until there is no more improvement in the objective function

In this notebook, I will be following the Google Developer Guide for OR-Tools, where I will be adding modifications and thoughts along the way.


## Contrainted Optimization

*Contraintes Optimization* is based on feasibility (finding a feasible solution) rather than optimization (finding an optimal solution) and focuses on the constraints and variables rather than the objective function. In fact, a CP problem may not even have an objective function — the goal may simply be to narrow down a vary large set of possible solutions to a more manageable subset by adding constraints to the problem.


## Integer Optimization

Problems that require some of the variables to be integers.


## Routing Problems


In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import pprint
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML 

pd.options.display.max_rows = 999
pd.options.display.width = 300
pd.options.display.max_columns = 500
start = time.time()

# Problems

## Stigler Diet Problem

[Link to Github Source](https://github.com/google/or-tools/blob/stable/examples/python/stigler_diet.py)

A Resource Allocation Problem. 

In [8]:
print("Define Table..")
# Nutrient minimums.
nutrients = [['Calories (kcal)', 3], ['Protein (g)', 70], [
    'Calcium (g)', 0.8
], ['Iron (mg)', 12], ['Vitamin A (KIU)', 5], ['Vitamin B1 (mg)', 1.8],
             ['Vitamin B2 (mg)', 2.7], ['Niacin (mg)',
                                        18], ['Vitamin C (mg)', 75]]

# Commodity, Unit, 1939 price (cents), Calories (kcal), Protein (g), Calcium (g), Iron (mg),
# Vitamin A (KIU), Vitamin B1 (mg), Vitamin B2 (mg), Niacin (mg), Vitamin C (mg)
data = [[
    'Wheat Flour (Enriched)', '10 lb.', 36, 44.7, 1411, 2, 365, 0, 55.4,
    33.3, 441, 0
], ['Macaroni', '1 lb.', 14.1, 11.6, 418, 0.7, 54, 0, 3.2, 1.9, 68, 0], [
    'Wheat Cereal (Enriched)', '28 oz.', 24.2, 11.8, 377, 14.4, 175, 0,
    14.4, 8.8, 114, 0
], ['Corn Flakes', '8 oz.', 7.1, 11.4, 252, 0.1, 56, 0, 13.5, 2.3, 68, 0], [
    'Corn Meal', '1 lb.', 4.6, 36.0, 897, 1.7, 99, 30.9, 17.4, 7.9, 106, 0
], [
    'Hominy Grits', '24 oz.', 8.5, 28.6, 680, 0.8, 80, 0, 10.6, 1.6, 110, 0
], ['Rice', '1 lb.', 7.5, 21.2, 460, 0.6, 41, 0, 2, 4.8, 60, 0], [
    'Rolled Oats', '1 lb.', 7.1, 25.3, 907, 5.1, 341, 0, 37.1, 8.9, 64, 0
], [
    'White Bread (Enriched)', '1 lb.', 7.9, 15.0, 488, 2.5, 115, 0, 13.8,
    8.5, 126, 0
], [
    'Whole Wheat Bread', '1 lb.', 9.1, 12.2, 484, 2.7, 125, 0, 13.9, 6.4,
    160, 0
], ['Rye Bread', '1 lb.', 9.1, 12.4, 439, 1.1, 82, 0, 9.9, 3, 66, 0], [
    'Pound Cake', '1 lb.', 24.8, 8.0, 130, 0.4, 31, 18.9, 2.8, 3, 17, 0
], ['Soda Crackers', '1 lb.', 15.1, 12.5, 288, 0.5, 50, 0, 0, 0, 0, 0], [
    'Milk', '1 qt.', 11, 6.1, 310, 10.5, 18, 16.8, 4, 16, 7, 177
], [
    'Evaporated Milk (can)', '14.5 oz.', 6.7, 8.4, 422, 15.1, 9, 26, 3,
    23.5, 11, 60
], ['Butter', '1 lb.', 30.8, 10.8, 9, 0.2, 3, 44.2, 0, 0.2, 2, 0], [
    'Oleomargarine', '1 lb.', 16.1, 20.6, 17, 0.6, 6, 55.8, 0.2, 0, 0, 0
], ['Eggs', '1 doz.', 32.6, 2.9, 238, 1.0, 52, 18.6, 2.8, 6.5, 1, 0], [
    'Cheese (Cheddar)', '1 lb.', 24.2, 7.4, 448, 16.4, 19, 28.1, 0.8, 10.3,
    4, 0
], ['Cream', '1/2 pt.', 14.1, 3.5, 49, 1.7, 3, 16.9, 0.6, 2.5, 0, 17], [
    'Peanut Butter', '1 lb.', 17.9, 15.7, 661, 1.0, 48, 0, 9.6, 8.1, 471, 0
], ['Mayonnaise', '1/2 pt.', 16.7, 8.6, 18, 0.2, 8, 2.7, 0.4, 0.5, 0, 0], [
    'Crisco', '1 lb.', 20.3, 20.1, 0, 0, 0, 0, 0, 0, 0, 0
], ['Lard', '1 lb.', 9.8, 41.7, 0, 0, 0, 0.2, 0, 0.5, 5, 0], [
    'Sirloin Steak', '1 lb.', 39.6, 2.9, 166, 0.1, 34, 0.2, 2.1, 2.9, 69, 0
], ['Round Steak', '1 lb.', 36.4, 2.2, 214, 0.1, 32, 0.4, 2.5, 2.4, 87, 0
   ], ['Rib Roast', '1 lb.', 29.2, 3.4, 213, 0.1, 33, 0, 0, 2, 0, 0], [
       'Chuck Roast', '1 lb.', 22.6, 3.6, 309, 0.2, 46, 0.4, 1, 4, 120, 0
   ], ['Plate', '1 lb.', 14.6, 8.5, 404, 0.2, 62, 0, 0.9, 0, 0, 0], [
       'Liver (Beef)', '1 lb.', 26.8, 2.2, 333, 0.2, 139, 169.2, 6.4, 50.8,
       316, 525
   ], [
       'Leg of Lamb', '1 lb.', 27.6, 3.1, 245, 0.1, 20, 0, 2.8, 3.9, 86, 0
   ], [
       'Lamb Chops (Rib)',
       '1 lb.', 36.6, 3.3, 140, 0.1, 15, 0, 1.7, 2.7, 54, 0
   ], [
       'Pork Chops', '1 lb.', 30.7, 3.5, 196, 0.2, 30, 0, 17.4, 2.7, 60, 0
   ], [
       'Pork Loin Roast',
       '1 lb.', 24.2, 4.4, 249, 0.3, 37, 0, 18.2, 3.6, 79, 0
   ], ['Bacon', '1 lb.', 25.6, 10.4, 152, 0.2, 23, 0, 1.8, 1.8, 71, 0], [
       'Ham, smoked', '1 lb.', 27.4, 6.7, 212, 0.2, 31, 0, 9.9, 3.3, 50, 0
   ], ['Salt Pork', '1 lb.', 16, 18.8, 164, 0.1, 26, 0, 1.4, 1.8, 0, 0], [
       'Roasting Chicken', '1 lb.', 30.3, 1.8, 184, 0.1, 30, 0.1, 0.9, 1.8,
       68, 46
   ], [
       'Veal Cutlets', '1 lb.', 42.3, 1.7, 156, 0.1, 24, 0, 1.4, 2.4, 57, 0
   ], [
       'Salmon, Pink (can)', '16 oz.', 13, 5.8, 705, 6.8, 45, 3.5,
       1, 4.9, 209, 0
   ], ['Apples', '1 lb.', 4.4, 5.8, 27, 0.5, 36, 7.3, 3.6, 2.7, 5, 544], [
       'Bananas', '1 lb.', 6.1, 4.9, 60, 0.4, 30, 17.4, 2.5, 3.5, 28, 498
   ], ['Lemons', '1 doz.', 26, 1.0, 21, 0.5, 14, 0, 0.5, 0, 4, 952], [
       'Oranges', '1 doz.', 30.9, 2.2, 40, 1.1, 18, 11.1, 3.6, 1.3, 10, 1998
   ], [
       'Green Beans', '1 lb.', 7.1, 2.4, 138, 3.7, 80, 69, 4.3, 5.8, 37, 862
   ], ['Cabbage', '1 lb.', 3.7, 2.6, 125, 4.0, 36, 7.2, 9, 4.5, 26, 5369], [
       'Carrots', '1 bunch', 4.7, 2.7, 73, 2.8, 43, 188.5, 6.1, 4.3, 89, 608
   ], ['Celery', '1 stalk', 7.3, 0.9, 51, 3.0, 23, 0.9, 1.4, 1.4, 9, 313], [
       'Lettuce', '1 head', 8.2, 0.4, 27, 1.1, 22, 112.4, 1.8, 3.4, 11, 449
   ], ['Onions', '1 lb.', 3.6, 5.8, 166, 3.8, 59, 16.6, 4.7, 5.9, 21,
       1184], [
           'Potatoes', '15 lb.', 34, 14.3, 336, 1.8, 118, 6.7, 29.4, 7.1,
           198, 2522
       ], [
           'Spinach', '1 lb.', 8.1, 1.1, 106, 0, 13.8, 918.4, 5.7, 13.8, 33,
           2755
       ], [
           'Sweet Potatoes', '1 lb.', 5.1, 9.6, 138, 2.7, 54, 290.7, 8.4,
           5.4, 83, 1912
       ], [
           'Peaches (can)', 'No. 2 1/2', 16.8, 3.7, 20, 0.4, 10, 21.5, 0.5,
           1, 31, 196
       ], [
           'Pears (can)', 'No. 2 1/2', 20.4, 3.0, 8, 0.3, 8, 0.8, 0.8, 0.8,
           5, 81
       ], [
           'Pineapple (can)', 'No. 2 1/2', 21.3, 2.4, 16, 0.4, 8, 2, 2.8,
           0.8, 7, 399
       ], [
           'Asparagus (can)', 'No. 2', 27.7, 0.4, 33, 0.3, 12, 16.3, 1.4,
           2.1, 17, 272
       ], [
           'Green Beans (can)', 'No. 2', 10, 1.0, 54, 2, 65, 53.9, 1.6, 4.3,
           32, 431
       ], [
           'Pork and Beans (can)', '16 oz.', 7.1, 7.5, 364, 4, 134, 3.5,
           8.3, 7.7, 56, 0
       ], [
           'Corn (can)', 'No. 2', 10.4, 5.2, 136, 0.2, 16, 12, 1.6, 2.7, 42,
           218
       ], [
           'Peas (can)', 'No. 2', 13.8, 2.3, 136, 0.6, 45, 34.9, 4.9, 2.5,
           37, 370
       ], [
           'Tomatoes (can)', 'No. 2', 8.6, 1.3, 63, 0.7, 38, 53.2, 3.4, 2.5,
           36, 1253
       ], [
           'Tomato Soup (can)', '10 1/2 oz.', 7.6, 1.6, 71, 0.6, 43, 57.9,
           3.5, 2.4, 67, 862
       ], [
           'Peaches, Dried', '1 lb.', 15.7, 8.5, 87, 1.7, 173, 86.8, 1.2,
           4.3, 55, 57
       ], [
           'Prunes, Dried', '1 lb.', 9, 12.8, 99, 2.5, 154, 85.7, 3.9, 4.3,
           65, 257
       ], [
           'Raisins, Dried', '15 oz.', 9.4, 13.5, 104, 2.5, 136, 4.5, 6.3,
           1.4, 24, 136
       ], [
           'Peas, Dried', '1 lb.', 7.9, 20.0, 1367, 4.2, 345, 2.9, 28.7,
           18.4, 162, 0
       ], [
           'Lima Beans, Dried', '1 lb.', 8.9, 17.4, 1055, 3.7, 459, 5.1,
           26.9, 38.2, 93, 0
       ], [
           'Navy Beans, Dried', '1 lb.', 5.9, 26.9, 1691, 11.4, 792, 0,
           38.4, 24.6, 217, 0
       ], ['Coffee', '1 lb.', 22.4, 0, 0, 0, 0, 0, 4, 5.1, 50,
           0], ['Tea', '1/4 lb.', 17.4, 0, 0, 0, 0, 0, 0, 2.3, 42, 0],
        ['Cocoa', '8 oz.', 8.6, 8.7, 237, 3, 72, 0, 2, 11.9, 40, 0], [
            'Chocolate', '8 oz.', 16.2, 8.0, 77, 1.3, 39, 0, 0.9, 3.4, 14, 0
        ], ['Sugar', '10 lb.', 51.7, 34.9, 0, 0, 0, 0, 0, 0, 0, 0],
        ['Corn Syrup', '24 oz.', 13.7, 14.7, 0, 0.5, 74, 0, 0, 0, 5, 0], [
            'Molasses', '18 oz.', 13.6, 9.0, 0, 10.3, 244, 0, 1.9, 7.5, 146,
            0
        ], [
            'Strawberry Preserves', '1 lb.', 20.5, 6.4, 11, 0.4, 7, 0.2,
            0.2, 0.4, 3, 0
        ]]

print("Nutritional Contraints Table")
display(pd.DataFrame(nutrients, columns = ['Nutrient', 'Daily Recommended Intake']).set_index('Nutrient'))
print("Food Nutrition Table")
display(pd.DataFrame(data,columns = [
    "Commodity",
    "Unit",
    "1939 price (cents)",
    "Calories",
    "Protein (g)",
    "Calcium (g)",
    "Iron (mg)",
    "Vitamin A (IU)",
    "Thiamine (mg)",
    "Riboflavin (mg)",
    "Niacin (mg)",
    "Ascorbic Acid (mg)"]).set_index("Commodity"))

Define Table..
Nutritional Contraints Table


,Daily Recommended Intake
Nutrient,
Calories (kcal),3.0
Protein (g),70.0
Calcium (g),0.8
Iron (mg),12.0
Vitamin A (KIU),5.0
Vitamin B1 (mg),1.8
Vitamin B2 (mg),2.7
Niacin (mg),18.0
Vitamin C (mg),75.0


Food Nutrition Table


,Unit,1939 price (cents),Calories,Protein (g),Calcium (g),Iron (mg),Vitamin A (IU),Thiamine (mg),Riboflavin (mg),Niacin (mg),Ascorbic Acid (mg)
Commodity,,,,,,,,,,,
Wheat Flour (Enriched),10 lb.,36.0,44.7,1411,2.0,365.0,0.0,55.4,33.3,441,0
Macaroni,1 lb.,14.1,11.6,418,0.7,54.0,0.0,3.2,1.9,68,0
Wheat Cereal (Enriched),28 oz.,24.2,11.8,377,14.4,175.0,0.0,14.4,8.8,114,0
Corn Flakes,8 oz.,7.1,11.4,252,0.1,56.0,0.0,13.5,2.3,68,0
Corn Meal,1 lb.,4.6,36.0,897,1.7,99.0,30.9,17.4,7.9,106,0
Hominy Grits,24 oz.,8.5,28.6,680,0.8,80.0,0.0,10.6,1.6,110,0
Rice,1 lb.,7.5,21.2,460,0.6,41.0,0.0,2.0,4.8,60,0
Rolled Oats,1 lb.,7.1,25.3,907,5.1,341.0,0.0,37.1,8.9,64,0
White Bread (Enriched),1 lb.,7.9,15.0,488,2.5,115.0,0.0,13.8,8.5,126,0


Note on the Food Nutrition Table, it is a bit misleading. While there is the unit and price information, all the other columns are nutrition levels scaled per dollar. This enables the objective function weight to be fixed at 1..

### Framing the Problem
Framing an optimization problem is a big part of the work. It is here where understanding about the objective and contraints must be materialized mathematically. The nature of these functions will dictate which solution to apply.

#### **Objective Function:** <br>
\begin{align}
f(minimize) & = \sum_{i} food[i] \cdot 1
\end{align}

where Food is the amount of money spent on foodstuff i.


#### **Contraints:** <br>
\begin{align}
\sum_{j} data[j][i+3] \cdot food[j] \geq nutrients[i][1]
\end{align}

Where data[j][i+3] is the calories per dollar of food j. and nutrients[i] is the minimal receommended intake value per nutrient i.

In [9]:
from six.moves import xrange
from ortools.linear_solver import pywraplp

# Instantiate a Glop solver, naming it LinearExample.
solver = pywraplp.Solver('StiglerDietExample',
                         pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

# Declare an array to hold our variables.
# Must be non-negative variables..
foods = [solver.NumVar(0.0, solver.infinity(), item[0]) for item in data]

# Objective function: Minimize the sum of (price-normalized) foods.
objective = solver.Objective()
# Here the Objective function looks something like food1 * 1 + food2 * 1 .. 
for food in foods:
    objective.SetCoefficient(food, 1)
# Minimization Problem
objective.SetMinimization()

# Create the constraints, one per nutrient.
constraints = []
for i, nutrient in enumerate(nutrients):
    # X * Nutrient >= 0
    constraints.append(solver.Constraint(nutrient[1], solver.infinity()))
    for j, item in enumerate(data):
        # Define all the Nutrients and Weights
        constraints[i].SetCoefficient(foods[j], item[i + 3])

print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())

Number of variables = 77
Number of constraints = 9


### Solving the Problem

In [10]:
# Solve the system.
status = solver.Solve()
# Check that the problem has an optimal solution.
if status != pywraplp.Solver.OPTIMAL:
    print("The problem does not have an optimal solution!")
    exit(1)

nutrients_result = [0] * len(nutrients)
nutrient_per_food = {}

print('')
print('Annual Foods:')
for i, food in enumerate(foods):
    if food.solution_value() > 0.0:
        print('{}: ${}'.format(data[i][0], 365. * food.solution_value()))
        
        for j, nutrient in enumerate(nutrients):
            # print(nutrient, data[i][j + 3] * food.solution_value())
            if food in nutrient_per_food:
                nutrient_per_food[food].append(data[i][j + 3] * food.solution_value())
            else:
                nutrient_per_food[food]=[data[i][j + 3] * food.solution_value()]
            
            # nutrient_per_food.append((data[i][0],nutrient,data[i][j + 3] * food.solution_value())) 
            
    for j, nutrient in enumerate(nutrients):
        nutrients_result[j] += data[i][j + 3] * food.solution_value()
        
        
print('')
print('Optimal annual price: ${:.4f}'.format(365. * objective.Value()))
print('')
print('Nutrients per day:')
for i, nutrient in enumerate(nutrients):
    print('{}: {:.2f} (min {})'.format(nutrient[0], nutrients_result[i],
                                       nutrient[1]))
print('')
print('Advanced usage:')
print('Problem solved in ', solver.wall_time(), ' milliseconds')
print('Problem solved in ', solver.iterations(), ' iterations')

foods_df = pd.DataFrame.from_dict(nutrient_per_food, orient='index', columns=[n[0] for n in nutrients])

for i, nutrient in enumerate(nutrients):
    foods_df[nutrient[0]]=(foods_df[nutrient[0]]/nutrients_result[i]*100).round(2)

display(foods_df)    


Annual Foods:
Wheat Flour (Enriched): $10.774457511918223
Liver (Beef): $0.6907834111074193
Cabbage: $4.093268864842877
Spinach: $1.8277960703546996
Navy Beans, Dried: $22.275425687243036

Optimal annual price: $39.6617

Nutrients per day:
Calories (kcal): 3.00 (min 3)
Protein (g): 147.41 (min 70)
Calcium (g): 0.80 (min 0.8)
Iron (mg): 59.84 (min 12)
Vitamin A (KIU): 5.00 (min 5)
Vitamin B1 (mg): 4.12 (min 1.8)
Vitamin B2 (mg): 2.70 (min 2.7)
Niacin (mg): 27.32 (min 18)
Vitamin C (mg): 75.00 (min 75)

Advanced usage:
Problem solved in  3226  milliseconds
Problem solved in  14  iterations


,Calories (kcal),Protein (g),Calcium (g),Iron (mg),Vitamin A (KIU),Vitamin B1 (mg),Vitamin B2 (mg),Niacin (mg),Vitamin C (mg)
Wheat Flour (Enriched),43.98,28.25,7.38,18.00,0.00,39.69,36.41,47.66,0.00
Liver (Beef),0.14,0.43,0.05,0.44,6.40,0.29,3.56,2.19,1.32
Cabbage,0.97,0.95,5.61,0.67,1.61,2.45,1.87,1.07,80.28
Spinach,0.18,0.36,0.00,0.12,91.98,0.69,2.56,0.60,18.39
"Navy Beans, Dried",54.72,70.01,86.97,80.77,0.00,56.87,55.60,48.48,0.00
